In [1]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 20.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 68.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1


In [2]:
!pip install dotenv

In [3]:
!pip install flask_cors

In [4]:
!pip install pyngrok

In [5]:
from flask import Flask, render_template, request, jsonify
import os
import torch
import json
import re
from dotenv import load_dotenv
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login

In [ ]:
import getpass

hugging_token = getpass.getpass("Enter your Hugging Face Token: ")
login(token=hugging_token)

base_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
lora_model_name = "jialeCharlotte/finbot"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="fp4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config, 
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, lora_model_name)

model.config.use_cache = False
model.config.sliding_window = None

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.37M [00:00<?, ?B/s]

In [7]:
import shutil

shutil.copytree('/kaggle/input/app/transformers/default/1/WebDeployment', '/kaggle/working/WebDeployment', dirs_exist_ok=True)

'/kaggle/working/WebDeployment'

In [8]:
from threading import Thread
from flask_cors import CORS

# Load environment variables
load_dotenv()

# Initialize Flask app
app = Flask(__name__, template_folder='/kaggle/working/WebDeployment/templates',
                      static_folder='/kaggle/working/WebDeployment/static')
CORS(app) 

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/analyze', methods=['POST'])
def analyze_sentiment():
    data = request.json
    title = data.get('title', '')
    description = data.get('description', '')
    ticker = data.get('ticker', '')
    
    # Create the prompt
    prompt = f"""You are a financial analyst in a leading hedge fund. 
Analyze the sentiment of the following financial news for the given stock ticker step by step.

Title: "{title}"
Summary: "{description}"
Stock Ticker: {ticker}

Step 1: Identify key financial terms and their implications.
Step 2: Determine whether the news suggests market optimism, pessimism, or neutrality for this specific stock.
Step 3: Based on your analysis, classify the sentiment into one of the following categories:
- "Bullish": If the news suggests confidence, growth, or positive impact on this stock.
- "Bearish": If the news suggests decline, risks, or negative impact on this stock.
- "Neutral": If the news is ambiguous or does not convey strong sentiment.

Finally, **return only** the final result in valid JSON format, with the structure:
{{
  "ticker": "{ticker}",
  "sentiment": "Bullish" | "Bearish" | "Neutral",
  "sentiment_reasoning": "Provide a brief explanation of the sentiment analysis."
}}

Do not include any extra text or explanations outside the JSON.
### Response:
(Return a valid JSON format only. Do not repeat the prompt.)
"""
    
    try:
        # Using the exact same generation code pattern as in Kaggle
        # Tokenize the prompt
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512, padding="max_length")
        
        # Move all tensors inside inputs to the correct device
        device_to_use = "cuda" if torch.cuda.is_available() else "cpu"
        inputs = {key: value.to(device_to_use) for key, value in inputs.items()}
        
        # Text generation
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=100,
                early_stopping=False,
                do_sample=True,
                num_return_sequences=1,
                num_beams=1,
                return_dict_in_generate=True
            )
        
        # Extract only the newly generated part
        generated_tokens = output.sequences[:, inputs["input_ids"].shape[-1]:]
        generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True).strip()
        
        print("Generated Text:\n", generated_text)
        
        # Use the exact same JSON extraction method as in Kaggle
        try:
            json_text = (re.search(r'\{.*?\}', generated_text, re.DOTALL)).group(0)    #full text 
        except Exception as e:
            json_text = (re.search(r'\{.*?\}', generated_text+'."}\n}\n', re.DOTALL)).group(0) 
        if json_text.count('"')>12:
            extracted_json = json.loads(json_text.rsplit('"', (json_text.count('"')-12)+1)[0]+'"}')
        else:
            extracted_json = json.loads(json_text)
            
        return jsonify({"success": True, "response": extracted_json})
        
    except Exception as e:
        error_message = str(e)
        print(f"Error during generation: {error_message}")
        return jsonify({"success": False, "error": error_message}), 500

def run_flask():
    host = "0.0.0.0"
    port = 5000
    print(f"Flask app is running at: http://{host}:{port}/")
    app.run(debug=True, use_reloader=False, host=host, port=port)

flask_thread = Thread(target=run_flask)
flask_thread.start()

Flask app is running at: http://0.0.0.0:5000/
 * Serving Flask app '__main__'
 * Debug mode: on


In [ ]:
ngrok_token = getpass.getpass("Enter your ngrok Token: ")

In [ ]:
!ngrok authtoken ngrok_token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [12]:
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://450b-34-70-183-155.ngrok-free.app" -> "http://localhost:5000"


In [13]:
!lsof -i :5000

COMMAND PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3  31 root  108u  IPv4  24870      0t0  TCP *:5000 (LISTEN)


In [14]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"http-5000-f7e3d0ee-9487-4d5d-8795-6f8564c74069","ID":"50fb4c5b66cd86c96b3832ced0c67946","uri":"/api/tunnels/http-5000-f7e3d0ee-9487-4d5d-8795-6f8564c74069","public_url":"https://450b-34-70-183-155.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [15]:
!curl -v https://91d5-34-45-61-148.ngrok-free.app/

*   Trying 3.22.30.40:443...
* Connected to 91d5-34-45-61-148.ngrok-free.app (3.22.30.40) port 443 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
*  CAfile: /etc/ssl/certs/ca-certificates.crt
*  CApath: /etc/ssl/certs
* TLSv1.0 (OUT), TLS header, Certificate Status (22):
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
* TLSv1.2 (IN), TLS header, Certificate Status (22):
* TLSv1.3 (IN), TLS handshake, Server hello (2):
* TLSv1.2 (IN), TLS header, Finished (20):
* TLSv1.2 (IN), TLS header, Supplemental data (23):
* TLSv1.3 (IN), TLS handshake, Encrypted Extensions (8):
* TLSv1.2 (IN), TLS header, Supplemental data (23):
* TLSv1.3 (IN), TLS handshake, Certificate (11):
* TLSv1.2 (IN), TLS header, Supplemental data (23):
* TLSv1.3 (IN), TLS handshake, CERT verify (15):
* TLSv1.2 (IN), TLS header, Supplemental data (23):
* TLSv1.3 (IN), TLS handshake, Finished (20):
* TLSv1.2 (OUT), TLS header, Finished (20):
* TLSv1.3 (OUT), TLS change cipher, Change cipher spec (1):
* TLSv1.2 (OUT),

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Generated Text:
 {"ticker": "AAPL", "sentiment": "Bullish", "sentiment_reasoning": "The article highlights the success of Apple's MacBook Air, which is the top selling laptop in the market. Apple has been a major player in the laptop industry, with its products being widely popular and successful, and the company's recent acquisition of the company has further boosted its market share."}


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Generated Text:
 {"ticker": "AAPL", "sentiment": "Bullish", "sentiment_reasoning": "The article highlights that Apple Inc. achieved record-breaking revenue in Q1 2025, exceeding analyst expectations by 15%."}
